In [136]:
from ReadWriteGDocs import write_gdoc, read_gdoc_range
# write_gdoc(data, ws_name, sht_name, row, col)
# read_gdoc_range(ws_name,sht_name="Sheet1",startrow=1,startcol=1, endrow=1, endcol=1)

import pandas as pd
import numpy as np

input_ws = ["NY Mini 2017 Score Input - Men's (Responses)", "NY Mini 2017 Score Input - Women's (Responses)"]
output_ws = ["NY Mini 2017 - 1st Round", "NY Mini 2017 - 2nd Round"]

Read data from Form Responses

In [137]:
# Whenever the spreadsheet.updated time is before the current time, then need to rerun everything after this point

c=read_gdoc_range(input_ws[1], "Form Responses 1", 1,2,1,14)
c=c[0]
c[9] = "Team 1 Score 2"
c[10] = "Team 2 Score 2"
c[11] = "Team 1 Score 3"
c[12] = "Team 2 Score 3"

f_in = pd.DataFrame(read_gdoc_range(input_ws[1], "Form Responses 1", 2,2, 100, 14), 
                    columns=c)
f_in.replace('', np.nan, inplace=True)
f_in.dropna(inplace=True, how='all')

m_in = pd.DataFrame(read_gdoc_range(input_ws[0], "Form Responses 1", 2,2, 100, 14), 
                    columns=c)
m_in.replace('', np.nan, inplace=True)
m_in.dropna(inplace=True, how='all')

m_in

,Court Number,Game Number,Have both team captains signed scoresheets?,Your initials,Notes,Team 1 Name,Team 2 Name,Team 1 Score,Team 2 Score,Team 1 Score 2,Team 2 Score 2,Team 1 Score 3,Team 2 Score 3
0,M3,6,Yes,LY,NaN,NY Brooklyn,Montreal Freemason,21,15,16,21,NaN,NaN
1,M3,8,Yes,LY,NaN,NY Panda Glass,Montreal Freemason,21,18,17,21,22,20


Write to corresponding GDocs

First, figure out what order to write the scores.
Then, write the scores.

In [183]:
from math import isnan

for index,row in m_in.iterrows():
    sheet = "Court " + str(row['Court Number'])
    game = int(row['Game Number'])
    
    T1 = row['Team 1 Name']
    
    scores = np.array([np.array([float(row['Team 1 Score']) , '-', '-', float(row['Team 2 Score'])]),
                      np.array(['-' ]),
                      np.array([float(row['Team 1 Score 2']), '-','-', float(row['Team 2 Score 2'])]),
                      np.array(['-' ]),
                      np.array([float(row['Team 1 Score 3']), '-','-', float(row['Team 2 Score 3'])])]
                      )
    
    #read game team name (first one only)
    
    team_names = read_gdoc_range(output_ws[0], sheet, 13+game, 3, 13+game, 3)
    
    if T1 != team_names:
        np.fliplr(scores)
        
    if isnan(float(row['Team 1 Score 3'])):
        scores = scores[0]+scores[1]
    else:
        scores = scores[0] + scores[1] + scores[2]
    
    write_gdoc([scores], output_ws[0], sheet, 13+game, 17)

ServerNotFoundError: Unable to find the server at accounts.google.com